# 2021120044_김형겸_hw2

## 요구사항 1

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split


class TitanicDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.FloatTensor(X)
    self.y = torch.LongTensor(y)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    target = self.y[idx]
    return {'input': feature, 'target': target}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str

### init
X = 입력 특성을 나타내는 데이터, Float
y = 타깃 레이블, Long

### len
데이터셋의 샘플 수 반환, X의 길이 반환

### getitem
idx의 해당하는 입력 특성과 타깃을 딕셔너리 형태로 반환

### str
데이터셋 정보 표현
데이터셋의 샘플 수, 입력 특성 데이터 shape, 타깃 레이블 shape
***

In [ ]:
class TitanicTestDataset(Dataset):
  def __init__(self, X):
    self.X = torch.FloatTensor(X)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    return {'input': feature}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str

### init
생성자에서 데이터의 특성 X를 입력 받아 FloatTensor를 이용하여 텐서화

### len
데이터셋의 샘플 수 반환, X의 길이 반환

### getitem
idx의 해당하는 입력 특성 샘플 반환

### str
데이터셋 정보 표현
데이터셋의 샘플 수, 입력 특성 데이터 shape
***

In [ ]:
def get_preprocessed_dataset():
    #1
    CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    #2
    all_df = pd.concat([train_df, test_df], sort=False)

    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    #3
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)

    #4
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

### 1
파일 경로를 설정하고 훈련, 테스트 데이터 경로 설정하여 가지고 옴

### 2
데이터의 전처리 과정 get_preprocessed_daraset_1~6까지의 함수를 이용

### 3
훈련, 테스트 데이터 분할


훈련 데이터
>데이터의 타겟 변수와 아닌 부분으로 나누어 데이터 저장


테스트 데이터
>타겟 변수 데이터 제외하고 데이터 저장

### 4
3에서 만든 훈련 데이터를 random_split을 사용하여 8:2로 분리하여 train_dataset, validation_dataset로 나누고 이렇게 만들어진 train_dataset, validation_dataset, test_dataset를 반환
***

In [ ]:
def get_preprocessed_dataset_1(all_df):
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df

### 데이터 전처리 과정 1
Pclass(티켓의 클래스), Fare(요금)을 이용하여 Pclass 별로 그룹화 하여 각 Pclass Fare의 평균값을 계산하고 Fare_mean 데이터프레임으로 저장


Fare 결측치(NaN)를 확인하고 격측치에 평균 값을 넣어서 결측지가 없게 데이터 처리


__결측치(데이터의 값이 없는 것, NaN)__


DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=None, indicator=False, validate=None)
>데이터베이스 스타일 조인을 사용하여 병합
>
>
>>__주의 : 두 키 열 모두에 키가 Null 값인 행이 포합되어 있으면 일치되어 예기치 않는 결과 발생 가능__

***

In [ ]:
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

### 데이터 전처리 2
이름 데이터의 있는 3개의 컬럼 분리하여 저장


성, 이름, 호칭으로 구성되어 있는 name을 각 컬럼으로 나누어 데이터 처리


pandas.concat(objs, *, axis=0, join='outer', ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=False, copy=None)
>특정 축을 이용하여 pasda 개체를 연결
***

In [ ]:
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

### 데이터 전처리 3
데이터 전처리 1과 마찬가지로 나이의 결측치 부분 채우는 과정


호칭별로 나이 중간값을 계산(반올림하여 정수로 표현)하고 결측치인 부분에 호칭에 맞게 계산된 중간값을 채워 데이터 처리


DataFrame.drop(labels=None, *, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise')
>행이나 열에서 지정된 레이블 삭제
***

In [ ]:
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

### 데이터 전처리 4
데이터 열 추가(병합 등) 및 삭제


family_num 추가
>Parch(부모, 자녀 수) + SibSp(형제, 배우자 수)

alone 추가
>family_num == 0일 경우 1로 설정

__fillna : 결측값 채우기, DataFrame.fillna(value=None, *, method=None, axis=None, inplace=False, limit=None, downcast=_NoDefault.no_default)__
>__meteod 부분(ffill, bfill, interpolate, reindex, asfreq, None)__
>
>
>>ffill : 마지막 유효한 관찰을 다음 유효한 관찰로 전파하여 값을 채움
>>
>>
>>bfill : 공백을 메우기 위해 다음 유효한 관찰을 사용하여 값을 채움
>>
>>
>>interpolate : 보간을 사용하여 채움
>>
>>
>>reindex : 개체를 새 인덱스에 맞춤
>>
>>
>>asfreq : TimeSeries를 지정된 빈도로 변환


불필요 부분 제거 : "PassengerId", "Name", "family_name", "name", "Ticket", "Cabin" 학습에 정보 제공 필요없는 부분 제거
***

In [ ]:
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df

### 데이터 전처리 5
값 축소 및 결측치 처리


honorific 값 축소
>"Mr", "Miss", "Mrs", "Master" 이외의 값 "other"로 대체
>
>
>>카디널리티(고유한 값의 수) 줄여 데이터셋 간소화 및 불필요한 복잡성 줄임


Embarked fillna함수 이용 결측치 missing로 채워 데이터 처리
***

In [ ]:
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
          le = le.fit(all_df[category_feature])
          all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

### 데이터 전처리 6
카테고리 문자열 값을 수치 값으로 변환


LabelEncoder : 카테고리 변수의 고유한 값들을 수치형으로 매핑
>ex)male, female => 0, 1
***

In [ ]:
from torch import nn
class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential(
      nn.Linear(n_input, 30),
      nn.ReLU(),
      nn.Linear(30, 30),
      nn.ReLU(),
      nn.Linear(30, n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

### 모델 정의
nn.Module, nn.Sequential 이용
>1번 레이어 : 입력 피처 수에서 30개 피처로 매칭
>
>
>2번 레이어 : 30개의 피처에서 30개의 피처로 매핑
>
>
>3번 레이어 : 30개의 피처에서 출력 수로 매핑


ReLU활성화 함수로 비선형성 도입
>ReLU6, ELU, LeakuReLU, PReLU 등으로 사용 가능


forword 함수
>PyTorch 모델에서 반드시 구현해야하는 메소드, 입력 데이터의 순방향 전파 정의
>
>
>모델의 출력 생성 후 반환
***

In [ ]:
def test(test_data_loader):
  print("[TEST]")
  batch = next(iter(test_data_loader))
  print("{0}".format(batch['input'].shape))
  my_model = MyModel(n_input=11, n_output=2)
  output_batch = my_model(batch['input'])
  prediction_batch = torch.argmax(output_batch, dim=1)
  for idx, prediction in enumerate(prediction_batch, start=892):
      print(idx, prediction.item())

### 학습 모델 test
아직 만들어 지지 않은 학습 모델을 이용하여 testdata를 돌려 결과값을 나타내는 함수


input(특성) 11, output 2(PassengerId,Survived)
***

In [ ]:
if __name__ == "__main__":
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

  print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
  ))
  print("#" * 50, 1)

  for idx, sample in enumerate(train_dataset):
    print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

  print("#" * 50, 2)

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  print("[TRAIN]")
  for idx, batch in enumerate(train_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("[VALIDATION]")
  for idx, batch in enumerate(validation_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("#" * 50, 3)

  test(test_data_loader)

### 실제 실행
위에서 만든 함수를 이용하여 데이터 셋을 불러오고 해당 데이터 셋을 로드하여 학습 후 테스트를 진행하는 부분으로 아직 학습하는 부분은 구현되어 있지 않은 코드


각각 출력값으로 데이터의 shape등을 확인할 수 있다
***

## 요구사항 2

### 추가 수정 코드

In [ ]:
import os
from datetime import datetime

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import wandb
import argparse


class TitanicDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.FloatTensor(X)
    self.y = torch.LongTensor(y)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    target = self.y[idx]
    return {'input': feature, 'target': target}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str


class TitanicTestDataset(Dataset):
  def __init__(self, X):
    self.X = torch.FloatTensor(X)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    return {'input': feature}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str


def get_preprocessed_dataset():
    CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))

    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    all_df = pd.concat([train_df, test_df], sort=False)

    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset


def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df


def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df


def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df


def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df


def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df


def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
          le = le.fit(all_df[category_feature])
          all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df


from torch import nn, optim


class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      nn.PReLU(),
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      nn.PReLU(),
      nn.Linear(40, 30),
      nn.PReLU(),
      nn.Linear(30, 20),
      nn.PReLU(),
      nn.Linear(20, 10),
      nn.PReLU(),
      nn.Linear(10, n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x


def get_model_and_optimizer():
  my_model = MyModel(n_input=11, n_output=2)
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

  return my_model, optimizer

def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  beforetrainingloss = 0.0
  beforevalidationloss = 0.0

  n_epochs = wandb.config.epochs
  loss_fn = nn.CrossEntropyLoss()  # Use a built-in loss function
  next_print_epoch = 100

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      output_train = model(train_batch['input'])
      loss = loss_fn(output_train, train_batch['target'])
      loss_train += loss.item()
      num_trains += 1

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        output_validation = model(validation_batch['input'])
        loss = loss_fn(output_validation, validation_batch['target'])
        loss_validation += loss.item()
        num_validations += 1

    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations
    })

    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
      )
      next_print_epoch += 100
    # if beforetrainingloss-0.001 < (loss_train / num_trains)<beforetrainingloss+0.001 and beforevalidationloss-0.001 < (loss_validation / num_validations)<beforevalidationloss+0.001:
    #   print(
    #     f"Training loss {loss_train / num_trains:.4f}, "
    #     f"Validation loss {loss_validation / num_validations:.4f}"
    #   )
    #   break
    # beforetrainingloss = loss_train / num_trains
    # beforevalidationloss = loss_validation / num_validations


def test(test_data_loader):
  print("[TEST]")
  batch = next(iter(test_data_loader))
  print("{0}".format(batch['input'].shape))
  my_model = MyModel(n_input=11, n_output=2)
  output_batch = my_model(batch['input'])
  prediction_batch = torch.argmax(output_batch, dim=1)
  list = []
  for idx, prediction in enumerate(prediction_batch, start=892):
      print(idx, prediction.item())
      list.append([idx, prediction.item()])
  df = pd.DataFrame(list, columns=['PassengerId', 'Survived'])
  df.to_csv('submission.csv', index=False, encoding='cp949')

if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--wandb", action=argparse.BooleanOptionalAction, default=False, help="True or False"
    )

    parser.add_argument(
        "-b", "--batch_size", type=int, default=512, help="Batch size (int, default: 512)"
    )

    parser.add_argument(
        "-e", "--epochs", type=int, default=100, help="Number of training epochs (int, default:1_000)"
    )

    args = parser.parse_args()

    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,
        'n_hidden_unit_list': [40, 40],
    }

    wandb.init(
        mode="online" if args.wandb else "disabled",
        project="my_model_training",
        entity="pkqwaszx",
        notes="My first wandb experiment",
        tags=["my_model", "titanic_dataset"],
        name=current_time_str,
        config=config
    )

    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

    linear_model, optimizer = get_model_and_optimizer()

    # print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    #     len(train_dataset), len(validation_dataset), len(test_dataset)
    # ))
    #
    # for idx, sample in enumerate(train_dataset):
    #   print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))


    train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    # print("[TRAIN]")
    # for idx, batch in enumerate(train_data_loader):
    #   print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))
    #
    # print("[VALIDATION]")
    # for idx, batch in enumerate(validation_data_loader):
    #     print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    wandb.watch(linear_model, log="all")

    training_loop(
        model=linear_model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader
    )

    print("#" * 50, 3)

    test(test_data_loader)
    wandb.finish()

### 설명
wandb 코드를 활용하고 더 나은 성능을 위해 Activation Function, 피처 수 변경하며 확인


2023-10-13_15-50-02 : RELU 피처수 30


2023-10-13_15-51-26 : RELU 피처수 20


2023-10-13_15-52-38 : RELU 피처수 40


2023-10-13_15-53-56 : RELU 피처수 50


2023-10-13_16-02-04 : ELU 피처 수 50


2023-10-13_16-04-44 : LeakyReLU 피처 수 50


2023-10-13_16-06-05 : PReLU 피처 수 50
***

### wandb 사이트
https://wandb.ai/pkqwaszx/my_model_training?workspace=user-pkqwaszx

## 요구사항 3

코드 수정하며 에포크 길이 조절 및 레이어 수 변경(+피처 내림차) 및 csv 파일 생성


2023-10-13_16-23-14 : PReLU 피처 수 50, 최솟값 나오게 추가 학습 진행


2023-10-13_16-26-28 : ReLU 피처 수 50, 최솟값 나오게 추가 학습 진행


2023-10-13_17-01-48 : PReLU 피처 수 30, kaggle 첫 제출


2023-10-13_17-17-28 : LeakyReLU 피처 수 30, kaggle 2 제출


2023-10-13_17-27-01 : PReLU 피처 수 30, 정지 조건 0.001


2023-10-13_17-29-47 : PReLU 피처 수 30, 정지 조건 0.005


2023-10-13_17-31-15 : PReLU 피처 수 30, 정지 조건 0.003 전부 죽음 kaggle 3 제출


2023-10-13_17-34-58 : PReLU 피처 수 30, 정지 조건 0.002


2023-10-13_17-37-33 : LeakyReLU 피처 수 30, 정지 조건 0.003


2023-10-13_17-39-23 : LeakyReLU 피처 수 30, 정지 조건 0.001 전부 다 살음


2023-10-13_17-41-06 : LeakyReLU 피처 수 30, 정지 조건 0.002 전부 다 살음


2023-10-13_17-45-01 : LeakyReLU 피처 수 30, 정지 조건 최초 에포크 내부에서 실행


2023-10-13_17-46-05 : PReLU 피처 수 30, 내부 정지 조건 0.001 정지 안함, 전부 살음


2023-10-13_17-49-52 : PReLU 피처 수 30, 내부 정지 조건 0.003 빨리 멈춤, 전부 살음


2023-10-13_17-52-35 : PReLU 피처 수 30 정지 안시킴 kaggle 4 제출


2023-10-14_20-23-34 : PReLU 레이어 수 증가, 피처 수 30 정지 안시킴


2023-10-14_20-32-51 : PReLU 레이어 수 증가, 피처 수 30 정지 200 kaggle 5 제출


2023-10-14_20-36-15 : PReLU 레이어 수 증가(레이블 내 피처 수 줄어들게 변경), 정지 안함 kaggle 6 제출


2023-10-14_20-38-41 : PReLU 레이어 수 증가(레이블 내 피처 수 줄어들게 변경), 정지 100 kaggle 7 제출


__위 과정으로 에포크의 길이가 불필요하게 많아 질 경우 loss값은 줄더라도 test에서 이상한 값을 출력하게 됨(전부 살거나 죽거나), 에포크는 100정도로도 충분하다고 판단됨__


__레이어의 경우에도 2개는 너무 적다고 판단되어 레이어 수 추가 및 레이어 내 피처 수를 내림차순으로 구성되게 코드 변경__
***

### wandb 사이트 (요구 사항 2와 동일)
https://wandb.ai/pkqwaszx/my_model_training?workspace=user-pkqwaszx

## 요구 사항 4

![kaggle_rank](https://github.com/ratatuwee/d_l_h_w/blob/main/%ED%99%94%EB%A9%B4%20%EC%BA%A1%EC%B2%98%202023-10-14%20210751.png?raw=true)

## 요구사항 5
wandb 사이트


https://wandb.ai/pkqwaszx/my_model_training?workspace=user-pkqwaszx
***

## 고찰

이번 과제를 진행하면서 딥러닝을 실제로 분석해보고 더 나은 성능을 보이기 위하여 여러가지 시도를 해보며 코드들을 자세히 이해할 수 있는 시간이였다. 아직은 실력이 부족하여 kaggle에서의 아주 높은 점수를 맞지 못하며 아쉬웠고 딥러닝에 대하여 더 자세히 알고싶다는 생각이 드는 과제였다. kaggle에서 높은 점수를 맞을 수 있을지 승부욕이 생겨 과제에서 요구하는 사항과 추가적으로 딥러닝의 다른 코드들을 살펴보고 코드를 보완해보았지만 어떠한 식으로 해야더 높은 점수를 기록할 수 있을지 아직은 잘 모르겠어 이 부분이 아쉽다고 생각하였다.